model with the projectional function being leisioned, purely learning from distinct instances

In [2]:
# first, test generative model on one dimensional graphs
import sys
sys.path.append('./HCM') # outdated due to abstraction learning implementation
import pandas as pd
import seaborn as sns
import numpy as np
from Learning import *
from CG1 import *

In [7]:
import pandas as pd
df = pd.read_csv('/Users/swu/Desktop/research/motif_learning/data/simonsays_ed/data.csv')

dfm = {}  # model dataframe
dfm['blockcollect'] = []
dfm['ID'] = []
dfm['condition'] = []
dfm['correctcollect'] = []
dfm['p'] = []
dfm['trialcollect'] = []
dfm['recall_likelihood'] = []

column_names = ['ID', 'condition', 'recallcollect', 'instructioncollect','trialcollect', 'correctcollect']
dfs = pd.DataFrame(columns=column_names)  # Note that there are now row data inserted.



seql = 12
len_train = 40
len_test = 8

def convert_sequence(seq,keyassignment):
    keyassignment = list(keyassignment)
    ka = [keyassignment[2],keyassignment[7],keyassignment[12],keyassignment[17],keyassignment[22],keyassignment[27]]
    proj_seq = [ka.index(item)+ 1 for item in seq]
    return proj_seq

def calculate_prob(chunk_record, cg):
    p = 1
    for key in list(chunk_record.keys()):# key is the encoding time
        p = p*cg.chunks[chunk_record[key][0][0]].count/np.sum([item.count for item in cg.chunks])
    return p

def convert_sequence_backward_to_key(seq, keyassignment):
    keyassignment = list(keyassignment)
    inst_seq = []
    ka = [keyassignment[2], keyassignment[7], keyassignment[12], keyassignment[17], keyassignment[22],
          keyassignment[27]]
    for i in range(0, 12):
        inst_seq.append(ka[int(seq[i,0,0])-1])
    return inst_seq



for sub in np.unique(list(df['ID'])):
    # initialize chunking part with specified parameters
    cg = CG1(DT=0.1, theta=0.996)

    for trial in range(1, len_train + 3*len_test+ 1):
        ins_seq = df[(df['ID'] == sub)].iloc[(trial-1)*seql:trial*seql, :][
            'instructioncollect']
        subj_recall =  df[(df['ID'] == sub)].iloc[(trial-1)*seql:trial*seql, :][
            'recallcollect']
        keyassignment = list(df[(df['ID'] == sub)].iloc[(trial-1)*seql:trial*seql, :][
            'keyassignment'])[0]
        condition = list(df[(df['ID'] == sub)].iloc[(trial-1)*seql:trial*seql, :][
            'condition'])[0]
        block = list(df[(df['ID'] == sub)].iloc[(trial-1)*seql:trial*seql, :][
            'blockcollect'])[0]
        
        proj_seq = convert_sequence(list(ins_seq),keyassignment)
        proj_seq = np.array(proj_seq).reshape([-1, 1, 1])
    
        cg, chunkrecord = hcm_learning(proj_seq, cg)  # with the rational chunk models, rational_chunk_all_info(seq, cg)
        recalled_seq,ps = recall(cg, firstitem=proj_seq[0,0,0])   
        keypress_recalled_seq = convert_sequence_backward_to_key(recalled_seq,keyassignment)


        p_seq = np.prod(ps)# evaluate the probability of a recalled sequence        
        proj_recalled_seq = convert_sequence(list(subj_recall),keyassignment)
        proj_recalled_seq = np.array(proj_recalled_seq).reshape([-1, 1, 1])
        
        cg, chunkrecord = hcm_learning(proj_recalled_seq, cg, learn = False, chunk = False)  # with the rational chunk models, rational_chunk_all_info(seq, cg)
        recall_likelihood = calculate_prob(chunkrecord, cg)# evaluate the probability of a sequence

        dfm['blockcollect'].append(block)
        dfm['ID'].append(sub)
        dfm['condition'].append(condition)
        dfm['correctcollect'].append(acc_eval1d(recalled_seq, proj_seq))
        dfm['p'].append(p_seq)
        dfm['trialcollect'].append(trial)
        dfm['recall_likelihood'].append(recall_likelihood)
        
        ins_list = ins_seq.tolist()
        print(keypress_recalled_seq, ins_list)
    
        for i in range(0, 12):
            if keypress_recalled_seq[i] == ins_list[i]:
                correctcollect = 1
            else: correctcollect = 0
            dfs = dfs.append({'ID': sub,
                            'condition': condition,
                            'keyassignment': keyassignment,
                            'recallcollect': keypress_recalled_seq[i],
                            'trialcollect': trial,
                            'instructioncollect': ins_list[i],
                            'correctcollect': correctcollect}, ignore_index=True)



dfm = pd.DataFrame.from_dict(dfm)
csv_save_directory = '/Users/swu/Desktop/research/motif_learning/data/simonsays_ed/simulation_data_model_transition_recall_leison_projectional_motif.csv'

dfm.to_csv(csv_save_directory, index=False, header=True)


dfs_csv_save_directory = '/Users/swu/Desktop/research/motif_learning/data/simonsays_ed/simulation_data_model_transition_recall_leison_projectional_motif_individualkey.csv'
dfs.to_csv(dfs_csv_save_directory, index=False, header=True)

accuracy is:  0.6666666666666666
['D', 'D', 'K', 'K', 'K', 'K', 'K', 'K', 'K', 'K', 'K', 'K'] ['D', 'D', 'D', 'K', 'K', 'D', 'D', 'K', 'K', 'K', 'K', 'D']
accuracy is:  0.5
['S', 'K', 'K', 'K', 'K', 'K', 'K', 'K', 'K', 'K', 'D', 'D'] ['S', 'S', 'S', 'K', 'K', 'S', 'S', 'K', 'K', 'K', 'K', 'S']
accuracy is:  0.8333333333333334
['D', 'D', 'D', 'K', 'K', 'K', 'D', 'K', 'K', 'K', 'K', 'K'] ['D', 'D', 'D', 'K', 'K', 'D', 'D', 'K', 'K', 'K', 'K', 'D']
accuracy is:  0.4166666666666667
['K', 'K', 'K', 'K', 'D', 'K', 'K', 'K', 'K', 'K', 'K', 'S'] ['K', 'K', 'K', 'S', 'S', 'K', 'K', 'S', 'S', 'S', 'S', 'K']
accuracy is:  0.3333333333333333
['S', 'S', 'K', 'K', 'K', 'K', 'S', 'K', 'S', 'K', 'K', 'S'] ['S', 'S', 'S', 'J', 'J', 'S', 'S', 'J', 'J', 'J', 'J', 'S']
accuracy is:  0.5
['K', 'K', 'S', 'S', 'S', 'S', 'K', 'K', 'K', 'K', 'K', 'K'] ['K', 'K', 'K', 'S', 'S', 'K', 'K', 'S', 'S', 'S', 'S', 'K']
accuracy is:  0.5
['K', 'K', 'K', 'K', 'D', 'D', 'K', 'K', 'K', 'K', 'K', 'K'] ['K', 'K', 'K', 'D', 

accuracy is:  0.5
['F', 'F', 'F', 'F', 'F', 'L', 'L', 'F', 'F', 'L', 'L', 'F'] ['F', 'L', 'F', 'F', 'L', 'F', 'L', 'F', 'L', 'F', 'L', 'L']
accuracy is:  0.5
['L', 'L', 'F', 'F', 'F', 'F', 'L', 'L', 'L', 'F', 'F', 'L'] ['L', 'L', 'L', 'F', 'F', 'L', 'L', 'F', 'F', 'L', 'F', 'F']
accuracy is:  0.5833333333333334
['L', 'L', 'F', 'F', 'F', 'F', 'L', 'L', 'L', 'F', 'F', 'F'] ['L', 'L', 'F', 'L', 'F', 'L', 'F', 'L', 'F', 'F', 'L', 'F']
accuracy is:  0.5
['F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F'] ['F', 'F', 'L', 'L', 'F', 'L', 'F', 'L', 'L', 'F', 'F', 'L']
accuracy is:  0.3333333333333333
['F', 'L', 'F', 'F', 'F', 'L', 'L', 'L', 'F', 'F', 'F', 'F'] ['F', 'F', 'F', 'L', 'L', 'F', 'L', 'F', 'F', 'L', 'L', 'L']
accuracy is:  0.5833333333333334
['L', 'D', 'D', 'D', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L'] ['L', 'D', 'D', 'L', 'D', 'D', 'L', 'D', 'L', 'D', 'L', 'L']
accuracy is:  0.5833333333333334
['L', 'D', 'L', 'K', 'L', 'K', 'L', 'K', 'L', 'K', 'L', 'K'] ['L', 'K', 'L', 'L', 

accuracy is:  0.75
['F', 'F', 'F', 'F', 'J', 'F', 'J', 'J', 'J', 'J', 'J', 'J'] ['F', 'F', 'F', 'J', 'J', 'F', 'F', 'J', 'J', 'J', 'J', 'F']
accuracy is:  0.3333333333333333
['J', 'J', 'J', 'J', 'J', 'F', 'F', 'J', 'J', 'J', 'J', 'J'] ['J', 'J', 'J', 'F', 'F', 'J', 'J', 'F', 'F', 'F', 'F', 'J']
accuracy is:  0.8333333333333334
['F', 'F', 'J', 'J', 'J', 'F', 'F', 'J', 'J', 'J', 'F', 'F'] ['F', 'F', 'F', 'J', 'J', 'F', 'F', 'J', 'J', 'J', 'J', 'F']
accuracy is:  0.6666666666666666
['J', 'J', 'F', 'F', 'J', 'J', 'J', 'J', 'F', 'F', 'J', 'J'] ['J', 'J', 'J', 'F', 'F', 'J', 'J', 'F', 'F', 'F', 'F', 'J']
accuracy is:  0.5
['F', 'F', 'J', 'J', 'J', 'J', 'J', 'F', 'F', 'J', 'J', 'J'] ['F', 'F', 'F', 'J', 'J', 'F', 'F', 'J', 'J', 'J', 'J', 'F']
accuracy is:  0.3333333333333333
['F', 'F', 'F', 'J', 'J', 'F', 'F', 'J', 'J', 'J', 'J', 'F'] ['F', 'J', 'F', 'F', 'F', 'J', 'J', 'F', 'J', 'J', 'F', 'J']
accuracy is:  0.4166666666666667
['F', 'F', 'F', 'F', 'F', 'J', 'J', 'J', 'J', 'J', 'F', 'F'] ['F',

accuracy is:  0.4166666666666667
['S', 'S', 'S', 'S', 'S', 'K', 'S', 'S', 'S', 'S', 'S', 'S'] ['S', 'S', 'S', 'K', 'K', 'S', 'S', 'K', 'K', 'K', 'K', 'S']
accuracy is:  0.5833333333333334
['K', 'K', 'K', 'K', 'K', 'K', 'K', 'K', 'K', 'S', 'S', 'S'] ['K', 'K', 'K', 'S', 'S', 'K', 'K', 'S', 'S', 'S', 'S', 'K']
accuracy is:  0.5833333333333334
['S', 'S', 'S', 'S', 'S', 'S', 'K', 'K', 'K', 'S', 'K', 'K'] ['S', 'S', 'S', 'K', 'K', 'S', 'S', 'K', 'K', 'K', 'K', 'S']
accuracy is:  0.6666666666666666
['K', 'K', 'S', 'S', 'S', 'S', 'K', 'K', 'S', 'S', 'S', 'S'] ['K', 'K', 'K', 'S', 'S', 'K', 'K', 'S', 'S', 'S', 'S', 'K']
accuracy is:  0.5833333333333334
['S', 'K', 'K', 'K', 'S', 'S', 'S', 'S', 'K', 'K', 'S', 'S'] ['S', 'S', 'S', 'K', 'K', 'S', 'S', 'K', 'K', 'K', 'K', 'S']
accuracy is:  0.5
['K', 'K', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S'] ['K', 'S', 'S', 'K', 'K', 'S', 'K', 'S', 'K', 'K', 'S', 'S']
accuracy is:  0.75
['K', 'K', 'K', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S'] ['K',

accuracy is:  0.3333333333333333
['S', 'F', 'S', 'F', 'F', 'S', 'F', 'D', 'J', 'J', 'J', 'F'] ['S', 'J', 'S', 'J', 'S', 'J', 'J', 'S', 'S', 'J', 'J', 'S']
accuracy is:  0.25
['J', 'J', 'S', 'S', 'J', 'J', 'F', 'F', 'F', 'J', 'F', 'J'] ['J', 'J', 'J', 'D', 'D', 'J', 'J', 'J', 'D', 'D', 'D', 'D']
accuracy is:  0.4166666666666667
['S', 'F', 'S', 'J', 'J', 'J', 'F', 'F', 'J', 'F', 'F', 'J'] ['S', 'F', 'S', 'S', 'F', 'S', 'S', 'F', 'F', 'F', 'S', 'F']
accuracy is:  0.6666666666666666
['L', 'L', 'K', 'K', 'K', 'K', 'K', 'K', 'K', 'L', 'K', 'L'] ['L', 'L', 'K', 'K', 'L', 'K', 'K', 'K', 'L', 'K', 'L', 'L']
accuracy is:  0.6666666666666666
['L', 'L', 'K', 'K', 'L', 'L', 'K', 'K', 'L', 'L', 'K', 'K'] ['L', 'K', 'K', 'K', 'L', 'L', 'L', 'K', 'K', 'L', 'L', 'K']
accuracy is:  0.6666666666666666
['L', 'L', 'K', 'L', 'L', 'K', 'K', 'L', 'L', 'K', 'L', 'L'] ['L', 'K', 'K', 'K', 'L', 'L', 'K', 'K', 'L', 'K', 'L', 'L']
accuracy is:  0.5833333333333334
['K', 'K', 'L', 'L', 'K', 'K', 'K', 'L', 'K', 'K', 

...

accuracy is:  0.5
['J', 'J', 'D', 'J', 'D', 'J', 'J', 'J', 'D', 'D', 'J', 'J'] ['J', 'J', 'D', 'J', 'J', 'D', 'D', 'J', 'J', 'D', 'D', 'D']
accuracy is:  0.4166666666666667
['D', 'J', 'J', 'D', 'J', 'D', 'J', 'J', 'J', 'D', 'D', 'J'] ['D', 'D', 'D', 'J', 'J', 'J', 'J', 'D', 'D', 'J', 'D', 'J']
accuracy is:  0.5833333333333334
['J', 'J', 'D', 'J', 'D', 'J', 'D', 'J', 'D', 'J', 'D', 'J'] ['J', 'J', 'J', 'D', 'D', 'J', 'J', 'D', 'D', 'D', 'D', 'J']
accuracy is:  0.4166666666666667
['D', 'J', 'J', 'D', 'J', 'D', 'J', 'J', 'J', 'D', 'D', 'J'] ['D', 'D', 'D', 'J', 'J', 'D', 'D', 'J', 'J', 'J', 'J', 'D']
accuracy is:  0.6666666666666666
['J', 'J', 'D', 'D', 'J', 'J', 'D', 'D', 'D', 'D', 'J', 'J'] ['J', 'J', 'J', 'D', 'D', 'J', 'J', 'D', 'D', 'D', 'D', 'J']
accuracy is:  0.5833333333333334
['D', 'D', 'J', 'J', 'D', 'J', 'D', 'J', 'D', 'J', 'J', 'J'] ['D', 'D', 'D', 'J', 'J', 'D', 'D', 'J', 'J', 'J', 'J', 'D']
accuracy is:  0.16666666666666666
['J', 'D', 'J', 'J', 'J', 'D', 'D', 'J', 'J', 'J', 

accuracy is:  0.4166666666666667
['D', 'D', 'D', 'K', 'K', 'D', 'D', 'K', 'K', 'K', 'K', 'K'] ['D', 'D', 'D', 'J', 'J', 'D', 'D', 'J', 'J', 'J', 'J', 'D']
accuracy is:  0.5833333333333334
['F', 'F', 'F', 'F', 'F', 'F', 'F', 'S', 'F', 'F', 'F', 'F'] ['F', 'F', 'F', 'S', 'S', 'F', 'F', 'S', 'S', 'S', 'S', 'F']
accuracy is:  0.5833333333333334
['F', 'F', 'S', 'S', 'F', 'F', 'F', 'S', 'S', 'F', 'F', 'S'] ['F', 'F', 'F', 'S', 'S', 'F', 'F', 'S', 'S', 'S', 'S', 'F']
accuracy is:  0.5833333333333334
['F', 'F', 'S', 'S', 'F', 'F', 'S', 'S', 'S', 'F', 'S', 'S'] ['F', 'F', 'F', 'S', 'S', 'F', 'F', 'S', 'S', 'S', 'S', 'F']
accuracy is:  0.5833333333333334
['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'F', 'S', 'S', 'S'] ['S', 'S', 'S', 'F', 'F', 'S', 'S', 'F', 'F', 'F', 'F', 'S']
accuracy is:  0.5
['S', 'S', 'F', 'F', 'S', 'S', 'F', 'F', 'S', 'S', 'F', 'F'] ['S', 'S', 'S', 'F', 'F', 'S', 'S', 'F', 'F', 'F', 'F', 'S']
accuracy is:  0.5833333333333334
['F', 'F', 'F', 'F', 'F', 'F', 'F', 'S', 'S', 'F', '

accuracy is:  0.4166666666666667
['S', 'S', 'S', 'S', 'J', 'J', 'J', 'J', 'J', 'S', 'S', 'S'] ['S', 'J', 'J', 'J', 'J', 'J', 'S', 'S', 'S', 'S', 'J', 'S']
accuracy is:  0.3333333333333333
['D', 'D', 'D', 'D', 'J', 'J', 'J', 'J', 'D', 'J', 'S', 'S'] ['D', 'J', 'J', 'J', 'J', 'J', 'D', 'D', 'D', 'D', 'J', 'D']
accuracy is:  0.16666666666666666
['S', 'D', 'D', 'D', 'D', 'D', 'J', 'J', 'J', 'J', 'D', 'S'] ['S', 'J', 'J', 'J', 'J', 'J', 'S', 'S', 'S', 'S', 'J', 'S']
accuracy is:  0.08333333333333333
['J', 'D', 'S', 'S', 'S', 'S', 'D', 'S', 'S', 'S', 'S', 'D'] ['J', 'K', 'K', 'K', 'K', 'K', 'J', 'J', 'J', 'J', 'K', 'J']
accuracy is:  0.5833333333333334
['K', 'D', 'D', 'D', 'D', 'D', 'S', 'S', 'S', 'S', 'D', 'S'] ['K', 'D', 'D', 'D', 'D', 'D', 'K', 'K', 'K', 'K', 'D', 'K']
accuracy is:  0.16666666666666666
['K', 'K', 'D', 'D', 'D', 'J', 'D', 'D', 'D', 'D', 'D', 'J'] ['K', 'J', 'J', 'J', 'J', 'J', 'K', 'K', 'K', 'K', 'J', 'K']
accuracy is:  0.08333333333333333
['K', 'K', 'D', 'D', 'D', 'J', 'D

accuracy is:  0.3333333333333333
['K', 'K', 'J', 'J', 'J', 'J', 'J', 'S', 'S', 'J', 'J', 'S'] ['K', 'K', 'K', 'K', 'S', 'S', 'S', 'K', 'S', 'S', 'K', 'S']
accuracy is:  0.5
['K', 'J', 'L', 'L', 'L', 'K', 'J', 'L', 'J', 'J', 'L', 'S'] ['K', 'J', 'K', 'J', 'K', 'K', 'J', 'K', 'J', 'J', 'K', 'J']
accuracy is:  0.3333333333333333
['S', 'S', 'S', 'S', 'J', 'J', 'S', 'S', 'S', 'J', 'J', 'S'] ['S', 'J', 'S', 'S', 'S', 'J', 'J', 'J', 'J', 'S', 'S', 'J']
accuracy is:  0.6666666666666666
['J', 'S', 'S', 'J', 'J', 'S', 'S', 'J', 'J', 'S', 'S', 'J'] ['J', 'S', 'J', 'J', 'J', 'S', 'S', 'S', 'S', 'S', 'J', 'J']
accuracy is:  0.6666666666666666
['J', 'K', 'K', 'J', 'J', 'J', 'J', 'J', 'L', 'J', 'J', 'L'] ['J', 'L', 'L', 'J', 'J', 'L', 'J', 'L', 'L', 'J', 'J', 'L']
accuracy is:  0.3333333333333333
['L', 'J', 'J', 'L', 'L', 'J', 'L', 'L', 'L', 'S', 'S', 'S'] ['L', 'K', 'K', 'L', 'L', 'K', 'K', 'K', 'L', 'L', 'K', 'L']
accuracy is:  0.3333333333333333
['J', 'J', 'J', 'S', 'S', 'J', 'J', 'S', 'S', 'J', '

accuracy is:  0.08333333333333333
['L', 'J', 'F', 'F', 'F', 'J', 'J', 'J', 'J', 'J', 'J', 'J'] ['L', 'D', 'D', 'D', 'D', 'D', 'L', 'L', 'L', 'L', 'D', 'L']
accuracy is:  0.5833333333333334
['L', 'F', 'F', 'F', 'J', 'J', 'J', 'L', 'L', 'L', 'L', 'D'] ['L', 'F', 'F', 'F', 'F', 'F', 'L', 'L', 'L', 'L', 'F', 'L']
accuracy is:  0.4166666666666667
['F', 'F', 'F', 'J', 'J', 'J', 'J', 'L', 'L', 'L', 'L', 'F'] ['F', 'J', 'J', 'J', 'J', 'J', 'F', 'F', 'F', 'F', 'J', 'F']
accuracy is:  0.4166666666666667
['D', 'D', 'J', 'J', 'J', 'L', 'L', 'L', 'L', 'D', 'D', 'J'] ['D', 'J', 'J', 'J', 'J', 'J', 'D', 'D', 'D', 'D', 'J', 'D']
accuracy is:  0.4166666666666667
['L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'D', 'D', 'J'] ['L', 'D', 'D', 'D', 'D', 'D', 'L', 'L', 'L', 'L', 'D', 'L']
accuracy is:  0.25
['F', 'F', 'J', 'J', 'J', 'L', 'L', 'L', 'L', 'L', 'L', 'L'] ['F', 'L', 'L', 'L', 'L', 'L', 'F', 'F', 'F', 'F', 'L', 'F']
accuracy is:  0.5
['D', 'L', 'L', 'L', 'L', 'L', 'F', 'F', 'F', 'J', 'J', 'J'] ['D'

In [8]:
list(ins_seq)

['F', 'F', 'F', 'L', 'L', 'F', 'L', 'F', 'F', 'L', 'L', 'L']